In [20]:
import nextbike
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [5]:
dfRaw = nextbike.io.read_file()
dfTrips = nextbike.io.createTrips(dfRaw)
dfTrips = nextbike.io.drop_outliers(dfTrips)
dfTrips.head(2)

,bNumber,sTime,eTime,duration,sLong,sLat,eLong,eLat,weekend,bType,sPlaceNumber,ePlaceNumber,durationInSec
0,11169,2019-01-20 01:55:00,2019-01-20 02:06:00,00:11:00,8.763266,50.795224,8.770358,50.804522,True,15,5173.0,5159.0,660
1,11169,2019-01-20 08:15:00,2019-01-20 11:39:00,03:24:00,8.770358,50.804522,8.770358,50.804522,True,15,5159.0,5159.0,12240


In [38]:
dfTrips['sMonth'] = dfTrips['sTime'].dt.month
dfTrips['weekday'] = dfTrips['sTime'].dt.dayofweek
dfTrips['minute'] = dfTrips['sTime'].dt.minute
dfTrips.dropna(inplace=True)
dfPred = []
xScaler = []
yScaler = []
X = []
Y = []
regressor = []
for hour in range(24):
    print('Hour: '+ str(hour))
    dfPred.append(dfTrips.loc[dfTrips['sTime'].dt.hour == hour])
    X.append(dfPred[hour][['weekday', 'sPlaceNumber', 'sMonth']].values)
    Y.append(dfPred[hour]['duration'].values/60)
    
    x_test, x_train, y_test, y_train = train_test_split(X[hour],Y[hour], test_size=0.3)
    sscaler = StandardScaler()
    x_train = sscaler.fit_transform(x_train)
    x_test = sscaler.transform(x_test)
    xScaler.append(sscaler)
    
    sscaler2 = StandardScaler()

    y_train = sscaler2.fit_transform(y_train.reshape(-1,1))
    y_test = sscaler2.transform(y_test.reshape(-1,1))
    yScaler.append(sscaler2)
    reg = RandomForestRegressor(n_jobs=-1,n_estimators=1000)
    #print(np.isnan(x_train,out=True))
    reg.fit(x_train,y_train)

    print(reg.score(x_test,y_test))
        

Hour: 0
0.3280449745905605
Hour: 1
0.09720811093158754
Hour: 2
0.3774608494994621
Hour: 3
0.37029683225815835
Hour: 4
0.2858413479041937
Hour: 5
0.15208207117911066
Hour: 6
0.0919265381259814
Hour: 7
0.08840902448502741
Hour: 8
0.11332485506689072
Hour: 9
0.10745780684596683
Hour: 10
0.11017707926273411
Hour: 11
0.10738854798987872
Hour: 12
0.10583270624030205
Hour: 13
0.060300188491672735
Hour: 14
0.0933865454692032
Hour: 15
0.06008208625811329
Hour: 16
0.08371633895557296
Hour: 17
0.06582112062941747
Hour: 18
0.07210989569369974
Hour: 19
0.10093915330708904
Hour: 20
0.11137531347278451
Hour: 21
0.1530718559259181
Hour: 22
0.19559253653941222
Hour: 23
0.3463564459668985
